In [1]:
import os
import requests
import glob
import xarray as xr
from datetime import datetime, timedelta
import warnings

# Ignore RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
# Base URL
base_url = "https://data.ceda.ac.uk/neodc/esacci/soil_moisture/data/daily_files/COMBINED/v08.1/"

# Directory where files will be saved
download_dir = r"D:/VUB/_data/esa_cci_sm"

In [ ]:
# Create the download directory if it does not exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Define the date range
start_date = datetime(2000, 1, 1)
end_date = datetime(2022, 12, 31)

# Iterate through each day in the date range
current_date = start_date
while current_date <= end_date:
    # Format the filename based on the date
    date_str = current_date.strftime("%Y%m%d000000")
    year_str = current_date.strftime("%Y")
    filename = f"ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-{date_str}-fv08.1.nc"
    
    # Complete URL for the file
    file_url = f"{base_url}{year_str}/{filename}"
    
    # Download the file
    response = requests.get(file_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        with open(os.path.join(download_dir, filename), 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {filename}", end="\r")
    else:
        print(f"Failed to download: {filename} (Status Code: {response.status_code})", end="\r")
    
    # Move to the next day
    current_date += timedelta(days=1)

#### Extract region of interest and merge

In [ ]:
#esa_cci_sm
files = glob.glob(r"D:\VUB\_data\esa_cci_sm\*.nc")
files.sort()

ds_roi_=list()
for file in files:
    with xr.open_dataset(file) as ds:
        ds_roi = ds['sm'].sel(lon=slice(32, 40), lat=slice(5.5, -1.0))
        print('appending', os.path.basename(file).split("-")[5], end='\r')
        ds_roi_.append(ds_roi)
        ds.close()

    ds_roi_.append(ds_roi)

ds_roi = xr.concat(ds_roi_, dim='time')

In [ ]:
ds_roi.name = 'sm'
ds_roi.to_netcdf(r"D:\VUB\_data\nc_files\esa_cci_sm_2000_2022.nc")

In [ ]:
out_dir=r"D:/VUB/_data/nc_files"
ds_combined.to_netcdf(os.path.join(out_dir, "esa_cci_sm_2000_2022.nc"))

In [ ]:
ds_roi['sm'].plot()

#### Carbon tracker CO2

In [2]:
# Base URL
base_url = "https://gml.noaa.gov/aftp/products/carbontracker/co2/molefractions/co2_total_monthly/"

# Directory where files will be saved
download_dir = r"D:/VUB/_data/NOAA_CO2"


In [5]:
# Create the download directory if it does not exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Define the date range
years = range(2000, 2023)
months = range(1, 13)
#2-digit month
months = [f"{month:02d}" for month in months]

# Iterate through each month in the date range
for year in years:
    for month in months:
    
        # Complete URL for the file
        file_url = f"{base_url}{'CT2022.molefrac_glb3x2_'}{year}{'-'}{month}.nc"

        file_name = f"{year}{'-'}{month}.nc"
        
        # Download the file
        response = requests.get(file_url)
        
        # Check if the request was successful
        if response.status_code == 200:
            with open(os.path.join(download_dir, file_name), 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_name}", end="\r")
        else:
            print(f"Failed to download: {file_name} (Status Code: {response.status_code})", end="\r")